In [272]:
import pandas as pd
# Imports CSV
df = pd.read_csv('../processed_data/constituency_results.csv')
# Reduces CSV to relevant election years
filtered_df = df[df['Election Year'].isin([2005, 2010, 2015, 2017, 2019])]
#Creates new index
filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)
# Pivots table for the first time
pivot_votes_df = filtered_df.pivot_table(
    index=['Election Year', 'Boundary Year', 'Constituency', 'Candidate','Party', 'new_index'],
    values='Votes',
    fill_value=0
).reset_index()

/tmp/ipykernel_408438/2972378557.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)


In [273]:
# Reduces CSV to relevant parties
parties_cleaned = pivot_votes_df.reset_index(drop=False)['Party'].apply(lambda x: x if x in ['The Brexit Party', 'Conservative', 'Green Party', 'Liberal Democrats', 'Labour', 'Plaid Cymru', 'SNP', 'UK Independence Party (UKIP)'] else 'Other')
pivot_votes_df['Party_cleaned'] = parties_cleaned


In [275]:
#Drops irrelevant columns
pivot_votes_df.drop(columns='Party', inplace=True)
pivot_votes_df.set_index('new_index', inplace=True)


In [276]:
# Makes second pivot
pivot_table = pd.pivot_table(pivot_votes_df,
                             index=['new_index'],
                             columns=['Party_cleaned'],
                             values='Votes',
                             aggfunc='sum',
                             fill_value=0)

In [278]:
# Renames columns to follow our style
rename_dict = {
    'The Brexit Party': 'BRX_ACTUAL_CONS',
    'Conservative': 'CON_ACTUAL_CONS',
    'Green Party': 'GRE_ACTUAL_CONS',
    'Liberal Democrats': 'LIB_ACTUAL_CONS',
    'Labour': 'LAB_ACTUAL_CONS',
    'Plaid Cymru': 'PLC_ACTUAL_CONS',
    'SNP': 'SNP_ACTUAL_CONS',
    'UK Independence Party (UKIP)': 'UKI_ACTUAL_CONS',
    'Other': 'OTH_ACTUAL_CONS'
}
pivot_table.rename(columns=rename_dict, inplace=True)

In [279]:
#Readds relevant columns
pivot_table['Election Year'] = pivot_table.index.str.split('_').str[0].astype(int)
pivot_table['Constituency'] = pivot_table.index.str.split('_').str[1]
pivot_table['Boundary Year'] = pivot_table.index.str.split('_').str[2]

In [282]:
# Reorders columns
columns_reordered = ['Constituency', 'Election Year', 'Boundary Year',
                     'CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS',
                     'LIB_ACTUAL_CONS', 'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS',
                     'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS', 'UKI_ACTUAL_CONS']
df_final = pivot_table[columns_reordered]

In [286]:
# Calculates total votes
df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',
                  'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS',
                  'UKI_ACTUAL_CONS']].sum(axis=1)

/tmp/ipykernel_408438/2241159542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',


In [288]:
# Switches to percentage
df_percentage = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',
                    'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS',
                    'UKI_ACTUAL_CONS']].div(df_final['total_votes'], axis=0) * 100

In [289]:
df_result = pd.concat([df_final[['Election Year', 'Constituency']], df_percentage], axis=1)


In [290]:
df_result.rename(columns={'Election Year': 'election_year'}, inplace=True)

In [291]:
# Imports national election results to concatenante
election_results_df = pd.read_csv('../processed_data/election_results_dataframe.csv')
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [292]:
# Merges national onto constituency election results
cons_result_comp = df_result.merge(election_results_df, on='election_year', how='left')

In [294]:
# Renames national result columns
rename_dict2 = {
    'BRX_ACTUAL_PERCENTAGE': 'BRX_NAT_RESULT',
    'CON_ACTUAL_PERCENTAGE': 'CON_NAT_RESULT',
    'GRE_ACTUAL_PERCENTAGE': 'GRE_NAT_RESULT',
    'LIB_ACTUAL_PERCENTAGE': 'LIB_NAT_RESULT',
    'LABOUR_ACTUAL_PERCENTAGE': 'LAB_NAT_RESULT',
    'PLC_ACTUAL_PERCENTAGE': 'PLC_NAT_RESULT',
    'SNP_ACTUAL_PERCENTAGE': 'SNP_NAT_RESULT',
    'UKI_ACTUAL_PERCENTAGE': 'UKI_NAT_RESULT',
    'OTH_PERCENTAGE': 'OTH_NAT_RESULT'
}
cons_result_comp.rename(columns=rename_dict2, inplace=True)


In [297]:
# Drops and renames columns
cons_result_comp.drop(columns=['Geography', 'Country'], inplace=True)
cons_result_comp.rename(columns={'CON_ACT_CONS': 'CON_ACTUAL_CONS'}, inplace=True)
cons_result_comp.rename(columns={'BRX_ACT_CONS': 'BRX_ACTUAL_CONS'}, inplace=True)

KeyError: "['Geography', 'Country'] not found in axis"

In [298]:
# Calculates bias score
for index, row in cons_result_comp.iterrows():
    bias_scores = {}
    # Loop through each party
    for party in ['CON', 'GRE', 'LAB', 'LIB', 'OTH', 'PLC', 'SNP', 'BRX', 'UKI']:
        # Calculate the absolute difference between the constituency result and national result for each party
        bias_scores[party] = (row[f'{party}_ACTUAL_CONS'] - row[f'{party}_NAT_RESULT'])
    # Assign the bias scores to the respective columns
    for party, score in bias_scores.items():
        cons_result_comp.loc[index, f'{party}_Bias_Score'] = score

In [263]:
# Filters datafra,e
biases_df = cons_result_comp.filter(like='Bias')

In [267]:
# Calculates bias per constituency
average_biases_per_constituency = biases_df.groupby(cons_result_comp['Constituency']).mean()

In [306]:
average_biases_per_constituency['CON_Bias_Score'].sort_values(ascending=False)


Constituency
South Holland and The Deepings    26.066973
Kensington and Chelsea            25.541860
Maldon                            25.390749
Staffordshire South               23.256404
Rayleigh and Wickford             23.239319
                                    ...    
Foyle                            -38.168436
Ulster Mid                       -38.181136
Belfast West                     -38.220536
Fermanagh and South Tyrone       -38.239784
Belfast North                    -38.239784
Name: CON_Bias_Score, Length: 731, dtype: float64

In [307]:
average_biases_per_constituency['LAB_Bias_Score'].sort_values(ascending=False)


Constituency
Liverpool Walton        45.947636
Knowsley                45.898622
Bootle                  42.636704
Liverpool West Derby    39.149282
East Ham                38.947483
                          ...    
Antrim South           -33.339512
Antrim North           -33.339512
Down South             -33.339512
Antrim East            -33.339512
Belfast West           -33.339512
Name: LAB_Bias_Score, Length: 731, dtype: float64

In [309]:
average_biases_per_constituency.to_csv('../processed_data/constituency_bias_dataframe.csv', index=False)